#### vision : 모델이 이미지를 입력으로 받아 이해하는 능력
- 사진설명, 영수증 분석, 표 읽기, 차트 해석...
- 전용모델 : gpt-image-1(이미지 이해, 생성)

- Response API
- Images API
- Chat completions API <-- 예전 방식

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# 웹에 있는 특정 이미지 해석
# https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRBeIzE6LsRhx9QBcf6u-_hOT4X_DLMA-GE9w&s

from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "이 이미지에 무엇이 보이나요?"},
                {
                    "type": "input_image",
                    "image_url": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRBeIzE6LsRhx9QBcf6u-_hOT4X_DLMA-GE9w&s",
                },
            ],
        }
    ],
)

print(response.output_text)

사진 속은 친구들로 보이는 여러 명의 여성이 파란 소파에 모여 함께 앉아 서로를 안아 주고 밝게 웃고 있는 모습입니다. 대다수는 흰 티를 입고 있으며, 방은 파스텔 톤의 아늑한 분위기이고 소파 옆에 분홍색 담요나 쿠션 같은 소품이 보입니다. 분위기는 다정하고 화기애애해 보입니다. 더 구체적인 부분이나 다른 관점으로 설명 도와드릴까요?


In [3]:
# 로컬 파일 해석
from openai import OpenAI
client = OpenAI()

with open("./image/bag.png", "rb") as fr:
    file1 = client.files.create(file=fr, purpose="vision")

print(file1.id)

file-YTxP2jj4BXP4urLXYw46yE


In [4]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "해당 이미지는 상품입니다. 설명해주세요.",
                },
                {
                    "type": "input_image",
                    "file_id": file1.id,
                },
            ],
        }
    ],
)

print(response.output_text)

이 이미지는 BEICHAO 브랜드의 여행용 또는 스포츠 백입니다. 색상은 군용 스타일의 다크 그린이며, 가방의 손잡이는 오렌지색과 검정색 조합으로 디자인되어 있습니다. 가방에는 여러 개의 지퍼 포켓이 있어 실용성이 높고, 안쪽에 충분한 공간이 있어 다양한 용도로 사용할 수 있습니다. 어깨 끈이 있어 편리하게 메고 다닐 수 있습니다.


In [ ]:
from openai import OpenAI
import base64

client = OpenAI()

# 이미지 생성 response API
response = client.responses.create(
    model="gpt-4.1-mini",  # vision 지원
    input="Generate an image of gray tabby cat hugging an otter with an orange scarf",
    tools=[{"type": "image_generation"}],
)

# 파일 저장
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_base64 = image_data[0]
    with open("./image/cat_and_otter.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-4.1-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [9]:
# 이미지 전용 생성 모델
from openai import OpenAI
import base64
from io import BytesIO
from PIL import Image

client = OpenAI()
response = client.images.generate(
    model="gpt-image-1-mini",
    prompt="A cute red panda reading a book, watercolor style",
    size="640x640",
    quality="high",
    output_format="png",
)

# 파일 저장
b64 = response.data[0].b64_json
img_bytes = base64.b64decode(b64)
img = Image.open(BytesIO(img_bytes)).convert("RGB")
img.save("./image/panda.png")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value: '640x640'. Supported values are: '1024x1024', '1024x1536', '1536x1024', and 'auto'.", 'type': 'invalid_request_error', 'param': 'size', 'code': 'invalid_value'}}